### 使用说明
1. 输入所需信息：
    * working file的文件名(带后缀)，如```PwQ_Pre-Allocation W34_Planners_WorkingFile - Copy.xlsm```，以及sheet_name如```2920```
    * excel属于你工作内容的起始行，如PIC为Yanqing的工作内容从excel中的第```8```行开始
    * 用于计算温饱线的excel文件名称(带后缀)，如```AGL needs W34.xlsx```，以及sheet_name如```Stock needed```
2. 运行本文件，在 [Cell] 中点击 Run All，会自动生成两个文件：
    * ```_pre_allocation_res.xlsx``` 储存计算结果
    * ```_pre_allocation_log.log```储存计算日志以信息查询与比对


In [1]:
# 注：使用前请先确保以下信息填写正确：
working_file_name = 'PwQ_Pre-Allocation W35_Planners_WorkingFile - YQ(skincare).xlsm'
working_file_sheet_name = '2920'
start_row = 8
agl_needs_file_name = 'AGL needs W34.xlsx'
agl_needs_file_sheet_name = 'Stock needed'
pic = 'Tang Ling'

In [2]:
import pandas as pd
import numpy as np
from pre_allocation import Solution
import logging
import xlsxwriter
from datetime import date
from openpyxl import load_workbook
from openpyxl.formula.translate import Translator

In [3]:
Log_Format = "%(levelname)s %(asctime)s - %(message)s"
today = date.today()
today = today.strftime("%Y-%m-%d")
file_name = today + '_pre_allocation'
logging.basicConfig(filename = str(today) + "_pre_allocation_log.log",
                    encoding = "utf-8",
                    filemode = "w",
                    format = Log_Format, 
                    level = logging.INFO)
logger = logging.getLogger()

In [4]:
working_file = pd.read_excel(working_file_name, sheet_name=working_file_sheet_name).iloc[5:, 1:]
working_file.columns = working_file.iloc[0]
working_file = working_file[working_file['PIC'] == pic]
working_file.index = np.arange(start_row, len(working_file) + start_row)
columns = ['Material', 'PIC', 'Prepack', 'SOH', 'Avg Ind.Req (M)','KR(1200) -', 'Direct KR', 'HK(1220) -', 'Direct HK','AU(1180) -', 'Direct AU', 'TW(1280) -', 'Direct TW', 'CN(1190) -',
       'Direct CN', 'JP(1230) -', 'Direct JP', 'TH(1340) -', 'Direct TH']
working_file = working_file[columns].replace(np.nan, 0)
working_file = working_file.astype({'Avg Ind.Req (M)':int})

In [5]:
working_file.head(20)

5,Material,PIC,Prepack,SOH,Avg Ind.Req (M),KR(1200) -,Direct KR,HK(1220) -,Direct HK,AU(1180) -,Direct AU,TW(1280) -,Direct TW,CN(1190) -,Direct CN,JP(1230) -,Direct JP,TH(1340) -,Direct TH
8,C024900001,Tang Ling,35,3938,550,0,0,0,0,35,0,0,0,0,0,1050,0,0,0
9,C024900100,Tang Ling,35,3465,176,0,0,0,0,35,0,0,0,0,0,630,0,0,0
10,C099600004,Tang Ling,24,817,34,0,0,0,0,48,0,0,0,0,0,0,0,0,0
11,C099600005,Tang Ling,30,281,42,90,0,0,0,0,0,0,0,0,0,0,0,120,0
12,C099600006,Tang Ling,30,460,70,0,0,0,0,30,0,0,0,0,0,0,0,60,0
13,C099600034,Tang Ling,25,1286,78,50,0,0,0,50,0,0,0,125,0,300,0,50,0
14,C099600163,Tang Ling,56,402,28,0,0,0,0,0,0,0,0,0,0,112,0,0,0
15,C099600229,Tang Ling,12,497,40,0,0,0,0,24,0,0,0,0,0,0,0,36,0
16,C099600265,Tang Ling,30,461,30,0,0,0,0,30,0,0,0,0,0,0,0,0,0
17,C099600295,Tang Ling,8,486,335,160,200,0,0,0,0,0,0,2240,0,0,80,288,0


In [6]:
newdf = working_file
newdf['KR(1200) -'] = np.where(newdf['KR(1200) -'] - newdf['Direct KR'] <= 0, 0, newdf['KR(1200) -'] - newdf['Direct KR'])
newdf['HK(1220) -'] = np.where(newdf['HK(1220) -'] - newdf['Direct HK'] <= 0, 0, newdf['HK(1220) -'] - newdf['Direct HK'])
newdf['AU(1180) -'] = np.where(newdf['AU(1180) -'] - newdf['Direct AU'] <= 0, 0, newdf['AU(1180) -'] - newdf['Direct AU'])
newdf['TW(1280) -'] = np.where(newdf['TW(1280) -'] - newdf['Direct TW'] <= 0, 0, newdf['TW(1280) -'] - newdf['Direct TW'])
newdf['CN(1190) -'] = np.where(newdf['CN(1190) -'] - newdf['Direct CN'] <= 0, 0, newdf['CN(1190) -'] - newdf['Direct CN'])
newdf['JP(1230) -'] = np.where(newdf['JP(1230) -'] - newdf['Direct JP'] <= 0, 0, newdf['JP(1230) -'] - newdf['Direct JP'])
newdf['TH(1340) -'] = np.where(newdf['TH(1340) -'] - newdf['Direct TH'] <= 0, 0, newdf['TH(1340) -'] - newdf['Direct TH'])
newdf['after DD'] = np.where(newdf['SOH'] - newdf['Avg Ind.Req (M)'] <= 0, 0, newdf['SOH'] - newdf['Avg Ind.Req (M)'])
newdf.drop(['Direct KR','Direct HK','Direct AU','Direct TW','Direct CN','Direct JP','Direct TH'], axis=1,inplace=True)

In [7]:
newdf.head(20)

5,Material,PIC,Prepack,SOH,Avg Ind.Req (M),KR(1200) -,HK(1220) -,AU(1180) -,TW(1280) -,CN(1190) -,JP(1230) -,TH(1340) -,after DD
8,C024900001,Tang Ling,35,3938,550,0,0,35,0,0,1050,0,3388
9,C024900100,Tang Ling,35,3465,176,0,0,35,0,0,630,0,3289
10,C099600004,Tang Ling,24,817,34,0,0,48,0,0,0,0,783
11,C099600005,Tang Ling,30,281,42,90,0,0,0,0,0,120,239
12,C099600006,Tang Ling,30,460,70,0,0,30,0,0,0,60,390
13,C099600034,Tang Ling,25,1286,78,50,0,50,0,125,300,50,1208
14,C099600163,Tang Ling,56,402,28,0,0,0,0,0,112,0,374
15,C099600229,Tang Ling,12,497,40,0,0,24,0,0,0,36,457
16,C099600265,Tang Ling,30,461,30,0,0,30,0,0,0,0,431
17,C099600295,Tang Ling,8,486,335,0,0,0,0,2240,0,288,151


In [8]:
skuList = list(working_file['Material'])
workbook = load_workbook(filename=agl_needs_file_name)
worksheet = workbook[agl_needs_file_sheet_name]
formula_row = 3
for sku in skuList:
    for col in ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I']:
        cell = col + str(formula_row)
        if col == 'A':
            worksheet[cell] = sku
        else:
            formula = worksheet[col+'3'].value
            worksheet[cell] = Translator(formula, origin=col+'3').translate_formula(cell)
    formula_row += 1 
workbook.save("agl_needs_updated.xlsx")

In [9]:
# 神奇bug，openpyxl保存的excel并未真正保存 -> https://codeantenna.com/a/iRcZiE5TVp
# 需要用win32com模拟打开并关闭excel来完成一次储存操作
# https://bbs.huaweicloud.com/blogs/363106 -> win32com安装，仅限windows
# pip install pypiwin32 或 pip install pywin32
# !!!以下命令仅用于windows系统，mac只能手动打开excel保存后关闭，然后跳过该代码继续执行

# 如需运行以下代码，取消注释即可

from win32com.client import Dispatch
def open_save(filename):
    xlApp = Dispatch('Excel.Application')
    xlApp.Visible = False
    xlBook = xlApp.Workbooks.Open(filename)
    xlBook.Save()
    xlBook.Close()
open_save(r"C:\Users\yawang\Desktop\lvmh_pre_allocation\agl_needs_updated.xlsx")

In [10]:
agl_needs = pd.read_excel("agl_needs_updated.xlsx", sheet_name=agl_needs_file_sheet_name).iloc[1:len(skuList)+2].fillna(0).rename({'SKU':'Material'}, axis=1)
agl_needs.drop(['Designation'], axis=1, inplace=True)
agl_needs = agl_needs.astype({'KR':int, 'HK': int, 'AU': int, 'TW':int, 'CN': int, 'JP': int, 'TH': int})
print(len(agl_needs))
agl_needs.head(20)

135


,Material,KR,HK,AU,TW,CN,JP,TH
1,C024900001,0,0,0,0,0,0,0
2,C024900100,0,0,0,0,0,0,0
3,C099600004,0,0,0,0,0,0,0
4,C099600005,0,0,0,0,0,0,0
5,C099600006,0,0,0,0,0,0,0
6,C099600034,-40,0,12,-178,-47,-44,-53
7,C099600163,0,0,0,0,0,0,0
8,C099600229,0,0,0,0,0,0,0
9,C099600265,0,0,0,0,0,0,0
10,C099600295,35,-55,0,0,-952,0,0


In [11]:
df = pd.merge(newdf, agl_needs, on='Material', how='outer')
df.drop(['SOH','Avg Ind.Req (M)'], axis=1, inplace=True)
df.insert(3, 'after DD', df.pop('after DD'))
df.index = np.arange(start_row, len(df) + start_row)
df.head(20)

,Material,PIC,Prepack,after DD,KR(1200) -,HK(1220) -,AU(1180) -,TW(1280) -,CN(1190) -,JP(1230) -,TH(1340) -,KR,HK,AU,TW,CN,JP,TH
8,C024900001,Tang Ling,35,3388,0,0,35,0,0,1050,0,0,0,0,0,0,0,0
9,C024900100,Tang Ling,35,3289,0,0,35,0,0,630,0,0,0,0,0,0,0,0
10,C099600004,Tang Ling,24,783,0,0,48,0,0,0,0,0,0,0,0,0,0,0
11,C099600005,Tang Ling,30,239,90,0,0,0,0,0,120,0,0,0,0,0,0,0
12,C099600006,Tang Ling,30,390,0,0,30,0,0,0,60,0,0,0,0,0,0,0
13,C099600034,Tang Ling,25,1208,50,0,50,0,125,300,50,-40,0,12,-178,-47,-44,-53
14,C099600163,Tang Ling,56,374,0,0,0,0,0,112,0,0,0,0,0,0,0,0
15,C099600229,Tang Ling,12,457,0,0,24,0,0,0,36,0,0,0,0,0,0,0
16,C099600265,Tang Ling,30,431,0,0,30,0,0,0,0,0,0,0,0,0,0,0
17,C099600295,Tang Ling,8,151,0,0,0,0,2240,0,288,35,-55,0,0,-952,0,0


In [12]:
def getFinalResult(row):
    prepack = row['Prepack']
    own = row['after DD']
    needs = [row['KR'], row['HK'], row['AU'], row['TW'], row['CN'], row['JP'], row['TH']]
    wants = [row['KR(1200) -'], row['HK(1220) -'], row['AU(1180) -'], row['TW(1280) -'], row['CN(1190) -'], row['JP(1230) -'], row['TH(1340) -']]
    s = Solution(needs, wants, prepack, own)
    res = s.getDistribution()
    logger.info(f"{row['Material']} ==> \n{s.explanation}\n")
    newCols = ['KR(1200) +', 'HK(1220) +', 'AU(1180) +', 'TW(1280) +', 'CN(1190) +', 'JP(1230) +', 'TH(1340) +', 'comments']
    for i in range(len(newCols)):
        if i == len(newCols) - 1:
            if s.ok:
                row[newCols[i]] = 'ok'
            else: 
                row[newCols[i]] = ''
        else:
            row[newCols[i]] = int(res[i])

    return row

In [13]:
res = df.apply(getFinalResult, axis=1)

In [14]:
res.head(20)

,Material,PIC,Prepack,after DD,KR(1200) -,HK(1220) -,AU(1180) -,TW(1280) -,CN(1190) -,JP(1230) -,...,JP,TH,KR(1200) +,HK(1220) +,AU(1180) +,TW(1280) +,CN(1190) +,JP(1230) +,TH(1340) +,comments
8,C024900001,Tang Ling,35,3388,0,0,35,0,0,1050,...,0,0,0,0,35,0,0,1050,0,ok
9,C024900100,Tang Ling,35,3289,0,0,35,0,0,630,...,0,0,0,0,35,0,0,630,0,ok
10,C099600004,Tang Ling,24,783,0,0,48,0,0,0,...,0,0,0,0,48,0,0,0,0,ok
11,C099600005,Tang Ling,30,239,90,0,0,0,0,0,...,0,0,90,0,0,0,0,0,120,ok
12,C099600006,Tang Ling,30,390,0,0,30,0,0,0,...,0,0,0,0,30,0,0,0,60,ok
13,C099600034,Tang Ling,25,1208,50,0,50,0,125,300,...,-44,-53,50,0,50,0,125,300,50,ok
14,C099600163,Tang Ling,56,374,0,0,0,0,0,112,...,0,0,0,0,0,0,0,112,0,ok
15,C099600229,Tang Ling,12,457,0,0,24,0,0,0,...,0,0,0,0,24,0,0,0,36,ok
16,C099600265,Tang Ling,30,431,0,0,30,0,0,0,...,0,0,0,0,30,0,0,0,0,ok
17,C099600295,Tang Ling,8,151,0,0,0,0,2240,0,...,0,0,0,0,0,0,144,0,0,


In [15]:
writer = pd.ExcelWriter('pre_allocation_res.xlsx', engine='xlsxwriter')
res.to_excel(writer, sheet_name='Sheet1')
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

bottom_row = str(len(res) + 1)
format_wants = ['#FFC7CE', 'F2:L' + bottom_row]
format_needs = ['#CCFFFF', 'M2:S' + bottom_row]
format_res = ['#E5FFCC', 'T2:Z' + bottom_row]
format_list = [format_wants, format_needs, format_res] # 3个涂色，wants, needs, res, 每个format信息为[color, left_top:right_bottom]
for each_format in format_list:
    fm = workbook.add_format({'bg_color': each_format[0]})
    area = each_format[1]
    worksheet.conditional_format(area, {'type': 'no_blanks','format': fm})
worksheet.freeze_panes(1,5)
writer.save()

In [16]:
# 似乎是windows特有的问题，如果不shutdown，无法重写或删除log日志
logging.shutdown()

#### 以下代码仅用于单测

In [17]:
prepack = 69
own = 47
needs = [0,0,-13,-37,-765,0,0]
wants = [0,0,16,0,828, 0, 0, 0]
s = Solution(needs, wants, prepack, own)
print(s.getDistribution())
print(s.explanation)
print(s.ok)

[0, 0, 0, 0, 0, 0, 0, 0]
1. 初次规整，prepack=69, 库存量=47, 各地区声明需求=[0, 0, 16, 0, 828, 0, 0, 0], 实际需求=[0, 0, 13, 37, 765, 0, 0], 温饱线=[0, 0, 13, 0, 765, 0, 0, 0], 当前贪婪线=[0, 0, 0, 0, 0, 0, 0, 0].
公平均等分配比例为[0, 0, 0.0114, 0, 0.6698, 0, 0, 0], 优化分配比例为[0, 0, 0, 0, 0, 0, 0, 0].
2. 库存量仅能先满足温饱线，最终分配表为[0, 0, 0, 0, 0, 0, 0, 0], 剩余库存47.

False
